In [ ]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
I = cv2.imread('Pictures/lego_14.jpg')
I_rgb = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)

if I is None:
    print("Error: Could not load image. Check file path and format.")
    exit()
    
plt.imshow(I_rgb)
plt.show()

In [ ]:
# Grayscale + blur
I_gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
I_blur = cv2.GaussianBlur(I_gray, (5,5), 0)

# Adaptive threshold
#Converted to black and white
B = cv2.adaptiveThreshold(
    I_blur, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY_INV,
    19, 2
)

# Morphology
kernel = np.ones((3,3), np.uint8)
#close = fill small holes
#open = removes small white noise outside objects
B_clean = cv2.morphologyEx(B, cv2.MORPH_CLOSE, kernel, iterations=2) 
B_clean = cv2.morphologyEx(B_clean, cv2.MORPH_OPEN, kernel, iterations=1)

# Find contours
contours, hierarchy = cv2.findContours(B_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours by area to get only LEGO blocks
min_area = 800
lego_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

print(f"Total number of LEGO blocks detected: {len(lego_contours)}")

# Create filled mask visualization
I_filled = np.ones_like(I_rgb) * 255  # white background
cv2.drawContours(I_filled, lego_contours, -1, (0, 0, 0), thickness=cv2.FILLED)

plt.figure(figsize=(12,6))
plt.subplot(1,3,1)
plt.imshow(I_rgb)
plt.title('Original Image')

plt.subplot(1,3,2)
plt.imshow(I_gray, cmap='gray')
plt.title("Gray Scale")

plt.subplot(1,3,3)
plt.imshow(I_filled)
plt.title(f"LEGO Blocks Filled Mask ({len(lego_contours)} blocks)")
plt.show()

In [ ]:
# Calculate grid size for displaying all blocks
num_blocks = len(lego_contours)
cols = 4  # Display 4 blocks per row
rows = (num_blocks + cols - 1) // cols  # Calculate needed rows
plt.figure(figsize=(15, 3*rows))

all_block_displays = [] #Added to save blocks in a list

#Looping through each LEGO block
for idx, contour in enumerate(lego_contours, 1):
    # Get bounding box
    x, y, w, h = cv2.boundingRect(contour)
    
    # Add padding
    padding = 5
    y_min = max(0, y - padding) #Don't go below 0
    y_max = min(I_rgb.shape[0], y + h + padding) #Image height cannot be exceeded
    x_min = max(0, x - padding) #Don't go left of 0
    x_max = min(I_rgb.shape[1], x + w + padding) #Image width cannot be exceeded
    
    # Create mask directly in the cropped region size
    mask_region = np.zeros((y_max - y_min, x_max - x_min), dtype=np.uint8)
    
    # Adjust contour coordinates to the cropped region
    contour_shifted = contour - [x_min, y_min]
    cv2.drawContours(mask_region, [contour_shifted], -1, 255, thickness=cv2.FILLED)
    
    # Extract the block region from original image
    block_original = I_rgb[y_min:y_max, x_min:x_max].copy()
    
    # Create a white background version
    block_display = np.ones_like(block_original) * 255
    block_display[mask_region > 0] = block_original[mask_region > 0]
    
    all_block_displays.append(block_display) #storing each display block

    # Display
    plt.subplot(rows, cols, idx)
    plt.imshow(block_display)
    plt.title(f"Block {idx}")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# HSV color ranges for LEGO colors
color_ranges = {
    "red":        ([0, 120, 70],   [10, 255, 255]),
    "red2":       ([170, 120, 70], [180, 255, 255]),  # for wrapping hue
    "orange":    ([11, 120, 100], [21, 255, 255]), 
    "yellow":    ([22, 120, 120], [37, 255, 255]),
    "lime":       ([36, 80, 80],   [55, 255, 255]),
    "green":      ([56, 80, 80],   [85, 255, 255]),
    "cyan":       ([86, 100, 100], [95, 255, 255]),
    "blue":       ([96, 100, 100], [125, 255, 255]),
    "purple":     ([126, 80, 80],  [155, 255, 255]),
    "pink":       ([156, 80, 120], [169, 255, 255]),
    "brown":      ([10, 100, 40],  [20, 255, 150]),
    "white":      ([0, 0, 200],    [180, 30, 255]),
    "gray":       ([0, 0, 60],     [180, 30, 200]),
    "black":      ([0, 0, 0],      [180, 255, 60])
    
}
# Map custom color names to valid display colors for matplotlib
plot_colors = {
    "red": "#FF0000",
    "red2": "#FF0000",
    "orange": "#FFA500",
    "yellow": "#FFFF00",
    "lime": "#99FF00",
    "green": "#008000",
    "cyan": "#00FFFF",
    "blue": "#0000FF",
    "purple": "#800080",
    "pink": "#FF69B4",
    "brown": "#8B4513",
    "white": "#FFFFFF",
    "gray": "#808080",
    "black": "#000000"
}


In [ ]:
# Convert full image to HSV once
I_hsv = cv2.cvtColor(I_rgb, cv2.COLOR_RGB2HSV)

#Loop through each LEGO block 
for idx, contour in enumerate(lego_contours, 1):
    # Bounding box with padding
    x, y, w, h = cv2.boundingRect(contour)
    padding = 5
    y_min = max(0, y - padding)
    y_max = min(I_rgb.shape[0], y + h + padding)
    x_min = max(0, x - padding)
    x_max = min(I_rgb.shape[1], x + w + padding)
    
    # Create mask for cropped region
    mask_region = np.zeros((y_max - y_min, x_max - x_min), dtype=np.uint8)
    contour_shifted = contour - [x_min, y_min]
    cv2.drawContours(mask_region, [contour_shifted], -1, 255, thickness=cv2.FILLED)
    
    # Extract block region
    block_original = I_rgb[y_min:y_max, x_min:x_max].copy()
    block_display = np.ones_like(block_original) * 255
    block_display[mask_region > 0] = block_original[mask_region > 0]
    
    # Convert to HSV
    block_hsv = I_hsv[y_min:y_max, x_min:x_max]
    
    # Count pixels for each color
    mask_counts = {}
    for color, (lower, upper) in color_ranges.items():
        lower = np.array(lower, np.uint8)
        upper = np.array(upper, np.uint8)
        mask = cv2.inRange(block_hsv, lower, upper)
        mask_block = cv2.bitwise_and(mask, mask, mask=mask_region)
        mask_counts[color] = cv2.countNonZero(mask_block)
    
    # Determine dominant color (ignore tiny counts)
    threshold = 50
    filtered_counts = {c: cnt for c, cnt in mask_counts.items() if cnt >= threshold}
    dominant_color = max(filtered_counts, key=filtered_counts.get) if filtered_counts else "unknown"
    
    # --- Display block ---
    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)
    plt.imshow(block_display)
    plt.title(f"Block {idx} ({dominant_color})")
    plt.axis('off')
    
    # --- Display color pixel counts as bar chart ---
    plt.subplot(1,2,2)
    colors = list(mask_counts.keys())
    counts = list(mask_counts.values())
    plt.bar(colors, counts, color=[plot_colors.get(c, "#CCCCCC") for c in colors])
    plt.xticks(rotation=45)
    plt.ylabel("Pixel count")
    plt.title("Color pixel counts")
    plt.tight_layout()
    plt.show()
    

In [ ]:
#Automatically detect lego pieces using countouring

# Invert so LEGO = white, background = black
B_inverted = cv2.bitwise_not(B_clean)

#Threshold to convert pixels less than 100 to 0 and pixels greater than or equal to 100 to 255 then inverting it
_, thresh = cv2.threshold(B_inverted, 100, 255, cv2.THRESH_BINARY_INV)

output = cv2.cvtColor(B_inverted, cv2.COLOR_GRAY2BGR)
piece_count = 0
# filtered_contour = []

#This loops through each lego piece that it was able to contour
for idx, contour in enumerate(lego_contours):

        x, y, w, h = cv2.boundingRect(contour)
        piece_count += 1

        #This creates a bounding box around the lego piece
        cv2.rectangle(output, (x,y), (x+w, y+h), (0,255,0), 2) #green rectangle 
        cv2.putText(output, f"Piece {piece_count}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2) #puts a label on the image of the piece number

        block_display = all_block_displays[idx] #This takes information from the previous code -its reusing code

        #Crops the grayscale image so that it can count the number of circles on a specific piece 
        lego_piece = I_gray[y:y+h, x:x+w]

        #Cleans the lego piece to reduce noise and improve contrast
        C_cleaned = cv2.medianBlur(lego_piece, 5)
        C_cleaned = cv2.equalizeHist(C_cleaned)

        #This detects circular shapes in the cropped piece
        circles = cv2.HoughCircles(
            C_cleaned,
            cv2.HOUGH_GRADIENT,
            dp=1.2,
            minDist=100,
            param1=150,
            param2=40,
            minRadius=10,
            maxRadius=60
        )
        
        #Checks if any circles were found or not
        if circles is not None:
            circles = np.uint16(np.around(circles))
            count = circles.shape[1] #counts the number or circles in the piece
            print(f"Piece {piece_count}: {count} circles detected")

            #Loops through each circle
            for (cx, cy, r) in circles[0, :]: 
                cv2.circle(output, (x + cx, y + cy), r, (0, 255, 0), 2) # draws the circle in green 

            #Writes text to count number of circles on that specific piece
            cv2.putText(output, f"Circles: {count}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        else: 
            print(f"Piece {piece_count}: No circles detected")


plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
plt.title("LEGO Pieces")
plt.show()


In [ ]:


summary = []  # store color, stud_count

# Re-run the dominant color + stud count classification but without plotting
for idx, contour in enumerate(lego_contours):

    # Bounding box
    x, y, w, h = cv2.boundingRect(contour)
    padding = 5
    y_min = max(0, y - padding)
    y_max = min(I_rgb.shape[0], y + h + padding)
    x_min = max(0, x - padding)
    x_max = min(I_rgb.shape[1], x + w + padding)

    # Mask region
    mask_region = np.zeros((y_max - y_min, x_max - x_min), dtype=np.uint8)
    contour_shifted = contour - [x_min, y_min]
    cv2.drawContours(mask_region, [contour_shifted], -1, 255, thickness=cv2.FILLED)

    # Extract HSV block
    block_hsv = I_hsv[y_min:y_max, x_min:x_max]

    # Count color pixels
    mask_counts = {}
    for color, (lower, upper) in color_ranges.items():
        lower = np.array(lower, np.uint8)
        upper = np.array(upper, np.uint8)
        mask = cv2.inRange(block_hsv, lower, upper)
        mask_block = cv2.bitwise_and(mask, mask, mask=mask_region)
        mask_counts[color] = cv2.countNonZero(mask_block)

    # Choose dominant color
    threshold = 50
    filtered = {c: n for c, n in mask_counts.items() if n >= threshold}
    dominant_color = max(filtered, key=filtered.get) if filtered else "unknown"

    #Count studs 
    lego_piece = I_gray[y:y+h, x:x+w]
    C_cleaned = cv2.medianBlur(lego_piece, 5)
    C_cleaned = cv2.equalizeHist(C_cleaned)

    circles = cv2.HoughCircles(
        C_cleaned,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=100,
        param1=150,
        param2=40,
        minRadius=10,
        maxRadius=60
    )

    stud_count = circles.shape[1] if circles is not None else 0

    summary.append((dominant_color, stud_count))

# Output summary
from collections import Counter

# group togther identical results
counts = Counter(summary)

print("\nYou have...\n")
for (color, studs), qty in counts.items():
    print(f"{qty} {color} brick(s) with {studs} stud(s)")
